In [1]:
# Trade data summary for ASX electricity contract

In [2]:
# Import libraries
import pandas as pd
import re

In [3]:
file_path = 'trades.csv'

In [ ]:
# Define the mappings from the ASX electricity contract cheat sheet
contract_codes = {
    'B': 'Quarter_base_futures', 'P': 'Quarter_base_futures', 'G': 'Quarter_$300_cap',
    'H': 'Base_strip', 'D': 'Peak_strip', 'R': 'Cap_strip',
    'E': 'Month_base_futures'
}

region_codes = {
    'N': 'NSW', 'Q': 'QLD', 'V': 'VIC', 'S': 'SA'
}

expiry_month_codes = {
    'F': 'JAN', 'G': 'FEB', 'H': 'MAR', 'J': 'APR', 
    'K': 'MAY', 'M': 'JUN', 'N': 'JUL', 'Q': 'AUG', 
    'U': 'SEP', 'V': 'OCT', 'X': 'NOV', 'Z': 'DEC'
}

quarter_codes = {
    'H': 'Q1', 'M': 'Q2', 'U': 'Q3', 'Z': 'Q4'
}

In [5]:
# Function to decode the product code to plain English
def decode(code):
    match = re.match(r'([A-Z])([A-Z])([A-Z])(\d{4})([A-Z]?)(\d{7})?', code)
    
    if match:
        contract = contract_codes.get(match.group(1), '')
        region = region_codes.get(match.group(2), '')
        expiry = quarter_codes.get(match.group(3), '')
        year = match.group(4)
        option_type = match.group(5)
        strike_price = match.group(6)
        
        if not contract:
            raise ValueError(f"Invalid contract code: {match.group(1)}")
        if not region:
            raise ValueError(f"Invalid region code: {match.group(2)}")
        if not expiry:
            raise ValueError(f"Invalid expiry code: {match.group(3)}")
        
        if option_type == "F":
            return f"{contract} {region} {expiry} {year}"
        elif option_type in ["C", "P"]:
            option = "Call option" if option_type == "C" else "Put option"
            strike_price_dollars = f"${int(strike_price) / 100:.2f}"
            return f"{contract} {region} {expiry} {year} {option} with a strike price of {strike_price_dollars}"
    
    return f"{code} - Invalid code"

In [6]:
# Function to encode plain English to product code
def encode(plain_english):
    try:
        parts = plain_english.split()
        
        if len(parts) == 4 or len(parts) == 8:
            contract = parts[0]
            region = parts[1]
            expiry = parts[2]
            year_code = parts[3]
            
            contract_code = next((k for k, v in contract_codes.items() if v == contract), None)
            region_code = next((k for k, v in region_codes.items() if v == region), None)
            expiry_code = next((k for k, v in quarter_codes.items() if v == expiry), None)
            
            if not contract_code:
                raise ValueError(f"Invalid contract code: {contract}")
            if not region_code:
                raise ValueError(f"Invalid region code: {region}")
            if not expiry_code:
                raise ValueError(f"Invalid expiry code: {expiry}")
            
            if len(parts) == 4:
                return f"{contract_code}{region_code}{expiry_code}{year_code}F"
            
            option_type = "C" if parts[4] == "Call" else "P"
            strike_price_cents = f"{int(float(parts[-1][1:]) * 100):07d}"
            return f"{contract_code}{region_code}{expiry_code}{year_code}{option_type}{strike_price_cents}"
        
        raise ValueError("Invalid input format.")
    
    except Exception as e:
        return f"Error: {str(e)}"

In [7]:
# Example usage of the above encode and decode functions
code = "BNH2022C0006000"
plain_english = "Cap QLD Q4 2025"

decoded = decode(code)
encoded = encode(plain_english)

print(f"Decoded: {decoded}")
print(f"Encoded: {encoded}")

Decoded: Base NSW Q1 2022 Call option with a strike price of $60.00
Encoded: GQZ2025F


In [8]:
def read_trade_data(file_path):
    # Skip the first line with the separator information
    df = pd.read_csv(file_path, skiprows=1)
    df['Date'] = pd.to_datetime(df['Date'])  # Convert Date column to datetime format
    return df

In [ ]:
# Calculate the number of days a trade is away from the contract expiry
def get_expiry_date(contract_code):
    # Decode the contract code
    decoded_info = decode(contract_code)
    if "Invalid code" in decoded_info:
        return decoded_info
    
    # Extract expiry quarter and year
    parts = decoded_info.split()
    expiry_quarter = parts[2]
    expiry_year = parts[3]
    
    # Define the last day of each quarter
    last_days = {
        'Q1': '31/03',
        'Q2': '30/06',
        'Q3': '30/09',
        'Q4': '31/12'
    }
    
    # Get the last day for the given quarter
    last_day = last_days.get(expiry_quarter)
    
    if last_day:
        # Combine with the given year and convert to datetime object
        return pd.to_datetime(f"{last_day}/{expiry_year}", format="%d/%m/%Y")
    else:
        return "Invalid quarter"

In [10]:
def process_trade_data(df, trade_code):
    # Filter the dataframe by the specified trade code
    filtered_df = df[df['Code'] == trade_code]

    # Group by date and calculate the required sums
    grouped_df = filtered_df.groupby('Date').agg({
        'Cleared Volume': 'sum',
        'Face Value': 'sum',
        'Volume x MWh': 'sum',
        'Code': 'first',
        'Product': 'first',
        'Location': 'first',
        'Trade Type': 'first',
        'Period': 'first',
        'Year': 'first'
    }).reset_index()

    # Rename the columns
    grouped_df.rename(columns={
        'Cleared Volume': 'Daily contracts traded',
        'Face Value': 'Daily dollars traded',
        'Volume x MWh': 'Daily MWh traded'
    }, inplace=True)

    # Calculate the daily volume weighted average price in $/MWh
    grouped_df['Daily average price'] = grouped_df['Daily dollars traded'] / grouped_df['Daily MWh traded']

    # Sort values by Date before calculating cumulative sums
    grouped_df.sort_values(by=['Date'], inplace=True)

    # Calculate the cumulative sums
    grouped_df['Cumulative MWh traded'] = grouped_df['Daily MWh traded'].cumsum()
    grouped_df['Cumulative dollars traded'] = grouped_df['Daily dollars traded'].cumsum()

    # Calculate the cumulative trade volume-weighted average price in $/MWh
    grouped_df['Cumulative average price'] = grouped_df['Cumulative dollars traded'] / grouped_df['Cumulative MWh traded']

    # Get the contract expiry date
    expiry_date = get_expiry_date(trade_code)

    # Generate a date range covering all dates from the min to max date in the data
    all_dates = pd.date_range(start=grouped_df['Date'].min(), end=grouped_df['Date'].max())

    # Forward fill missing values until the last day of trade
    grouped_df = grouped_df.set_index('Date').reindex(all_dates).fillna(method='ffill')

    # Reset the index to make the date column a regular column again
    grouped_df = grouped_df.reset_index().rename(columns={'index': 'Date'})

    # Calculate the number of days of the trade from contract expiry
    grouped_df['Days from expiry'] = grouped_df['Date'].apply(lambda x: (x - expiry_date).days)

    return grouped_df


In [11]:
df = read_trade_data(file_path)

In [12]:
all_trade_codes = df['Code'].unique()

In [13]:
result_df = process_trade_data(df, all_trade_codes[0])
print(result_df)

        Date  Daily contracts traded  Daily dollars traded  Daily MWh traded  \
0 2018-10-16                      25              876000.0            219000   

              Code Product Location Trade Type Period  Year  \
0  HVM2022P0006200  Option      VIC  On Screen    Fin  2022   

   Daily average price  Cumulative MWh traded  Cumulative dollars traded  \
0                  4.0                 219000                   876000.0   

   Cumulative average price  Days from expiry  
0                       4.0             -1353  


In [14]:
# Initialize an empty DataFrame to store the final results
final_df = pd.DataFrame()

# Loop through each trade code in the list of interest
for trade_code in all_trade_codes:
    # Call the process_trade_data function and get the result for the current trade code
    result_df = process_trade_data(df, trade_code)
    
    # Append the result_df to the final_df
    final_df = pd.concat([final_df, result_df], ignore_index=True)

# Now final_df contains the appended results for all trade codes


ValueError: Invalid contract code: E

In [ ]:
final_df.to_csv('all_trades_summary.csv')